# Gotchas!

BioSTEAM reduces the time needed to develop and evaluate a production process by simplifying pressure handling and by providing optional modeling assumptions and a range of alternative specifications. New users may not expect some of these "BioSTEAM" behaviours. This chapter lists common "gotchas" so that you won't have to scratch your brain too hard. 

## Mixer valves assumption

When streams at different pressures are mixed, BioSTEAM assumes valves reduce the pressure of inlet streams to prevent backflow:

In [1]:
import biosteam as bst
from biorefineries.sugarcane import chemicals
bst.settings.set_thermo(chemicals)

# Mix streams at different pressures
s_in1 = bst.Stream('s_in1', Water=1, units='kg/hr',P=2*101325)
s_in2 = bst.Stream('s_in2', Water=1, units='kg/hr', P=101325)
M1 = bst.Mixer('M1', ins=[s_in1, s_in2], outs='s_out')
M1.simulate()
M1.outs[0].show() # Note how the outlet stream pressure is the minimum inlet stream pressure

Stream: s_out from <Mixer: M1>
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): Water  0.111


## Optional vapor-liquid equilibrium.

Mixing streams with different phases does not perform vapor-liquid equilibrium (VLE) by default:

In [2]:
s_in2.phase = 'g'
s_in2.T = 380
M1.simulate()
M1.outs[0].show() # Notice how the outlet is liquid and above the boiling point (energy balance is conserved)

Stream: s_out from <Mixer: M1>
 phase: 'l', T: 580.36 K, P: 101325 Pa
 flow (kmol/hr): Water  0.111


For rigorous VLE during mixing, set `rigorous=True`:

In [3]:
M1.rigorous = True
M1.simulate()
M1.outs[0].show()

MultiStream: s_out from <Mixer: M1>
 phases: ('g', 'l'), T: 373.12 K, P: 101325 Pa
 flow (kmol/hr): (g) Water  0.0894
                 (l) Water  0.0216


Heat exchangers also don't perform VLE by default:

In [4]:
H1 = bst.HXutility(
    'H1', ins=bst.Stream('s1', Water=1000, units='kg/hr'), outs='s2', T=400
)
H1.simulate()
H1.outs[0].show()

Stream: s2 from <HXutility: H1>
 phase: 'l', T: 400 K, P: 101325 Pa
 flow (kmol/hr): Water  55.5


Set `rigorous=True` for rigorous VLE during:

In [5]:
H1.rigorous = True
H1.simulate()
H1.outs[0].show()

MultiStream: s2 from <HXutility: H1>
 phases: ('g', 'l'), T: 400 K, P: 101325 Pa
 flow (kmol/hr): (g) Water  55.5


In [6]:
# No vapor-liquid equilibrium is performed (but energy balance is maintained)

### Simplified pressure handling

Pressure drops of unit operations are not accounted for due to the tremendous amount of detail required to do so (e.g., specifying pipe fittings and dimensions). Ultimately, we just want to estimate pump sizes and motor requirements. As a preliminary assumption, we can specify the design pressure gain that a pump must operate, `dP_design`, to estimate these requirements:

In [7]:
s3 = bst.Stream('s3', Water=1000, units='kg/hr')
P1 = bst.Pump('P1', ins=s3, outs='s4', P=101325, dP_design=4 * 101325)
P1.simulate()
# Note how, although inlet and outlet pressures are the same, the pump is assummed to work at a 4 atm pressure gain
P1.outs[0].show() 
P1.results()

Stream: s4 from <Pump: P1>
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): Water  55.5


Pump                               Units           P1
Power               Rate              kW        0.321
                    Cost          USD/hr       0.0251
Design              Ideal power       hp        0.151
                    Flow rate        gpm         4.42
                    Efficiency                  0.352
                    Actual power                 0.43
                    Pump power                    0.5
                    N                               1
                    Head              ft          386
                    Type                  Centrifugal
Purchase cost       Pump             USD     4.32e+03
                    Motor            USD          311
Total purchase cost                  USD     4.63e+03
Utility cost                      USD/hr       0.0251

### Broadcasting, slicing, and indexers

Indexers and chemical groups are very powerful tools for faster development in BioSTEAM, but may lead to some confusion. In BioSTEAM, all flow rate data is stored in mol, but we can easily access mass flow rate data through MassFlowProperty objects which reference molar data:

In [8]:
s5 = bst.Stream('s5', Water=1000, units='kg/hr')
Water_mol = s5.imol['Water']
Water_mass = s5.imass['Water']
# Note how the molar flow rate is a float while the mass flow rate is MassFlowProperty
print(Water_mol, type(Water_mol))
print(Water_mass, type(Water_mass))

55.508435061791985 <class 'numpy.float64'>
<Water: 1000 kg/hr> <class 'thermosteam.indexer.MassFlowProperty'>


In [9]:
s5.imol['Water'] = 2
print(Water_mass) # Note how the MassFlowProperty object is aware of changes to the molar water flow rate.

<Water: 36.031 kg/hr>


In [10]:
s5.imass['Water', 'Ethanol'] # Note how property arrays also reference the molar flow rate data.

property_array([<Water: 36.031 kg/hr>, <Ethanol: 0 kg/hr>])

Chemical groups (e.g., the "fiber" group defined below) can be used to get and set "lumped" chemical flow rates in a stream. However, they are not linked to the stream's flow rate data:

In [11]:
# Define a lignocellulose chemical group with composition
# 25, 47, and 28 wt. % lignin, cellulose, and hemicellulose, respectively
chemicals.define_group(
    name='fiber', 
    IDs=['Lignin', 'Cellulose', 'Hemicellulose'],
    composition=[0.25, 0.47, 0.28], 
    wt=True,
)

s6 = bst.Stream('s6', Water=1, fiber=1, units='kg/hr')
s6.show()

Stream: s6
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): Water          0.0555
                 Cellulose      0.0029
                 Hemicellulose  0.00212
                 Lignin         0.00164


In [12]:
fiber_mass = s6.imass['fiber'] 
print(fiber_mass, type(fiber_mass)) # Note how it is a float, not a MassFlowProperty

1.0 <class 'numpy.float64'>


In [13]:
print(s6.imass['fiber', 'Water']) # All data indexed with chemical groups are also floats

[1. 1.]


In [14]:
s6.imass['fiber', 'Water'] = [100, 100] 
s6.show() # Note how we can still set flow rates this way

Stream: s6
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): Water          5.55
                 Cellulose      0.29
                 Hemicellulose  0.212
                 Lignin         0.164


When chemical groups are used to define chemicals splits (e.g., at a splitter), they work as nested indices with broadcasting:

In [15]:
s7 = bst.Stream('s7', Water=1, fiber=1, units='kg/hr')
S1 = bst.Splitter('S1', ins=s7, split=dict(Water=0.1, fiber=0.5))
S1.isplit.show() # Note how the split is broadcasted to all components

SplitIndexer:
 Water          0.1
 Cellulose      0.5
 Hemicellulose  0.5
 Lignin         0.5


In [16]:
S1.isplit['fiber'] # Note how the splits for each component is returned (not the sum)

array([0.5, 0.5, 0.5])

In [17]:
S1.isplit['fiber', 'Water'] # Note how the splits are nested

array([array([0.5, 0.5, 0.5]), 0.1], dtype=object)

In [18]:
S1.isplit['fiber'] = [0.2, 0.5, 0.7] # This is also valid
S1.isplit.show()

SplitIndexer:
 Water          0.1
 Cellulose      0.5
 Hemicellulose  0.7
 Lignin         0.2
